<a href="https://colab.research.google.com/github/AlxaRu/NeuroGen_HW3/blob/main/NeuroGen_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dan Udi Shefler - 324959147
# Alexandra Rukban - 203615752
!pip install pysam

### import libraries, input parameters, initilize dictionaries and arrays
import pysam
from  scipy.stats import binom
import numpy as np
import matplotlib.pyplot as plt
import re

### start of input
# quality score 30 corresponds to 1/1000 error rate, or 1e-3 sequencing error
minimum_qaulity_score = 30
P = 1e-3 #  propability of sequencing error given qaulity score of 30
max_mapping_score = 42 # the maximal mapping quality means unique alignment, with bowtie2 this number is 42
p_cutoff = 0.05 # the cutoff for significance, note that bonferroni correction is used below
bam_file_to_read = "aligned_RNAreads_sorted.bam"
output_file = "editing_sites_spectrin.txt"
length_of_read = 100
total_orf_size = 16607506
### end of input

# initilize dictionary for counts per letter
counts_per_letter = {}

# initilize dictionary for type of modification
number_of_modifications = {}
type_of_modifications = ['AC','AG','AT','CA','CG','CT','GA','GC','GT','TA','TC','TG']
for x in type_of_modifications:
    number_of_modifications[x] = 0

# initilize numpy array for position of mismatch in the read
position_of_mismatch_inside_the_read = np.zeros(length_of_read)

# initilize numpy array for position of match in the read
position_of_match_inside_the_read = np.zeros(length_of_read)

# Create a dictionary to store Amino Acid changes
AAChanges = {'Synonymous': 0}

# Create a codon table dictionary to convert sequence to acid name
codon_table = {
    'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
    'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
    'UAU': 'Y', 'UAC': 'Y', 'UAA': '*', 'UAG': '*',
    'UGU': 'C', 'UGC': 'C', 'UGA': '*', 'UGG': 'W',
    'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
    'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
    'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
    'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}

In [ ]:
# Function definitions
def find_codon(sequence, changePos, originalSeq, change):
    # Make sure that we're using RNA bases
    if originalSeq == 'T':
        originalSeq = 'U'
    if change == 'T':
        change = 'U'
    # Find the start of the codon
    codon_start = (changePos // 3) * 3
    # Extract the original codon
    original_codon = sequence[codon_start:codon_start + 3].replace('T', 'U')
    codonLst = list(original_codon)
    codonLst[changePos % 3] = originalSeq
    original_codon = ''.join(codonLst)

    if len(codonLst) != 3:
        print(codonLst)
    # Create a list from the original codon for mutable string
    changed_codon_list = codonLst
    # Update the specific base in the codon
    changed_codon_list[(changePos % 3)] = change
    # Convert list back to string
    changed_codon = ''.join(changed_codon_list)

    return original_codon, changed_codon

In [ ]:
### use pysam to examine all the reads aligned for each position of each transcript and detect editing sites
samfile = pysam.AlignmentFile(bam_file_to_read, "rb") # read the indexed and sorted bam file using pysam
file2 = open(output_file, "w") # file in which the detected editing sited will be listed

# Read the transcriptome and output relevant data
pattern = re.compile(
    r'>(comp[^\s]+).*?OrfStart\s+(\d+)\s+OrfEnd\s+(\d+).*?\n([ATCG\n]+)',
    re.DOTALL
)

# Open the data file and read its content
with open ('pealeii.txt', 'r') as file:
    data = file.read()

# Find all matches in the data
matches = pattern.findall(data)

# Create a dictionary to store the name of the gene, their transcripts and orf ranges (start and end)
orf_dictionary = {}
for match in matches:
    gene_name, orf_start, orf_end, transcipt = match
    orf_start = int(orf_start)
    orf_end = int(orf_end)

    # Remove newlines from the transcripts
    transcipt = transcipt.replace('\n', '')

    # Store the transcript and the range of the transcript in the dictionary
    orf_dictionary[gene_name] = [transcipt, [orf_start, orf_end]]

for transcript_name in orf_dictionary.keys():
    # run over all transcripts
    file2.write("\nname of transcript = %s\n" % transcript_name) # list the current transcript
    # run over all positions of the transcript and fetch the reads aligned to it
    # only fetch reads that have the maximal mapping score (i.e. unique alignment)
    for pileupcolumn in samfile.pileup(transcript_name,min_mapping_quality = max_mapping_score):
        position = pileupcolumn.pos
        # Make sure the positions are in our reading frame according to pealeii.txt
        if position >= orf_start and position <= orf_end:
            # initilize the dictionary counts_per_letter
            type_of_letters=['A','C','G','T']
            for x in type_of_letters:
                counts_per_letter[x] = 0
            # run over all reads that align against the transcript in the given position
            for pileupread in pileupcolumn.pileups:
                # query position is None if is_del or is_refskip is set
                if not pileupread.is_del and not pileupread.is_refskip:
                    # demand minimum quality score for the aligned base
                    if pileupread.alignment.query_qualities[pileupread.query_position] >= minimum_qaulity_score:
                        # detect the base
                        letter = pileupread.alignment.query_sequence[pileupread.query_position]
                        # count how many times each base was detected
                        counts_per_letter[letter] = counts_per_letter[letter] + 1
            # sort the counts per base to detect the base most frequent and the second base i.e. the modification
            sorted_dict = dict(sorted(counts_per_letter.items(),
                                    key=lambda item: item[1],
                                    reverse=True)) # sort descending
            i = 0 # a running number to detect the first and second most frequent bases
            N = 0 # total number of reads aligned to the first and second most frequent bases
            for key, value in sorted_dict.items():
                i = i + 1
                if i == 1:
                    N = N + value
                    first_letter = key # the most frequent base
                elif i == 2:
                    X = value # the number of times the modified base appeared
                    N = N + value
                    second_letter = key # the second most frequent base
                    modification = first_letter + second_letter # the modification, for example AG (i.e A changed to G)
                else:
                    break
            # given a rate of sequencing error P, total number of counts N, and modified counts X \
            # the probability of event being a sequencing error is 1 - binomial_CDF(X - 1, N , P) \
            # this is the same as binomial_CDF(N - X , N, 1 - P) (but the latter is more accurate due to rounding errors)
            prob = binom.cdf(N - X, N, 1 - P)
            # bonferroni correction p_cutoff divided by the overall number of tests
            if prob < (p_cutoff / total_orf_size):
                # print the total number of reads aligned for the specfic location inside the transcript
                file2.write("\ncoverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
                # sort the number of bases and print them to file
                for key, value in sorted_dict.items():
                    file2.write('\ncounts of base %s = %s' % (key,str(value)))
                file2.write('\nX is %s and N is %s' % (str(X),str(N))) # print X and N, as defined above
                part1='\nP-value for sequencing error is\t'
                part2="{:.2e}".format(prob)
                part3='\n'
                file2.write(part1 + part2 + part3) # print the P-value
                startIdx = orf_dictionary[transcript_name][1][0] - 1
                endIdx = orf_dictionary[transcript_name][1][1]
                if (position - 1) in range(startIdx, endIdx):
                    original, new = find_codon(orf_dictionary[transcript_name][0][startIdx:endIdx],
                                               position - startIdx - 1,
                                               list(sorted_dict.keys())[0], list(sorted_dict.keys())[1])
                    original, new = codon_table[original], codon_table[new]
                    if (original != new):
                        file2.write('Amino Acid change: %s to %s\n' % (original, new))
                        if original + new in AAChanges.keys():
                            AAChanges[original + new] += 1
                        else:
                            AAChanges[original + new] = 1
                    else:
                        file2.write('Amino Acid unchanged: %s\n' % original)
                        AAChanges['Synonymous'] += 1
                else:
                    file2.write('Detected change not in ORF\n')

                # update the number of modifications
                number_of_modifications[modification]=number_of_modifications[modification] + 1

                # check the location of the match and mismatch in the read
                #  to estimate if the modifications came from the reads ends
                # run over all reads that align against the transcript in the given position
                for pileupread in pileupcolumn.pileups:
                    # query position is None if is_del or is_refskip is set
                    if not pileupread.is_del and not pileupread.is_refskip:
                        # demand minimum quality score for the aligned base
                        if pileupread.alignment.query_qualities[pileupread.query_position] >= minimum_qaulity_score:
                            # detect the base
                            letter = pileupread.alignment.query_sequence[pileupread.query_position]
                            if (letter == first_letter): # these are reads that show the match
                                position_of_match_inside_the_read[pileupread.query_position] = \
                                position_of_match_inside_the_read[pileupread.query_position] + 1
                            elif (letter == second_letter): # these are reads that show the mismatch
                                position_of_mismatch_inside_the_read[pileupread.query_position] = \
                                position_of_mismatch_inside_the_read[pileupread.query_position] + 1
                break
        else:
            print("Position ", position, " in gene ", transcript_name, " is out of range. ")
samfile.close()
file2.close()

FileNotFoundError: [Errno 2] could not open alignment file `aligned_RNAreads_sorted.bam`: No such file or directory

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### plot the results
# plot the number of modifications per modification type sorted from highest to lowest
sorted_dict = dict( sorted(number_of_modifications.items(),
                        key = lambda item: item[1],
                        reverse = True)) # sort descending
plt.bar(range(len(sorted_dict)), list(sorted_dict.values()), align = 'center')
plt.xticks(range(len(sorted_dict)), list(sorted_dict.keys()))
plt.ylabel('Number of modifications')
plt.xlabel('Modification type')
plt.show()
# plot the number of mismatches per position inside the read from base 1 to end
plt.plot(list(range(0, length_of_read)),\
         position_of_mismatch_inside_the_read / sum(position_of_mismatch_inside_the_read),\
         label = "mismatch")
# on the same figure, plot the number of matches per position inside the read from base 1 to end
plt.plot(list(range(0, length_of_read)),\
         position_of_match_inside_the_read / sum(position_of_match_inside_the_read),\
         label = "match")
plt.ylabel('Fraction')
plt.xlabel('LocationInsideTheRead')
plt.legend()
plt.show()

# Sort amino changes dictionary by values and print
AAChanges = dict(sorted(AAChanges.items(), key=lambda item: item[1], reverse=True))
for key in AAChanges:
    print(key, ": ", AAChanges[key])
samfile.close()